# Assignment 3: Machine Translation with seq2seq models
### Name : Atharv Subhekar
### CWID : 20015840


In [ ]:
# Importing required librarie
from datasets import load_dataset
from transformers import MarianMTModel, MarianTokenizer, M2M100ForConditionalGeneration, M2M100Tokenizer,MBartForConditionalGeneration, MBart50TokenizerFast
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk.tokenize import word_tokenize
import numpy as np
import sacrebleu
import statistics

## Data Preprocessing


In [ ]:
# Specifying the dataset name using the language codes
dataset = load_dataset("Muennighoff/flores200", 'eng_Latn-spa_Latn')

In [ ]:
# Loading the dataset and storing english and spanish sentences
english = dataset['dev']['sentence_eng_Latn']
english_100 = english[:100]
english_20 = english[:20] # Using smaller dataset for easier computation

spanish = dataset['dev']['sentence_spa_Latn']
spanish_100 = spanish[:100]
spanish_20 = spanish[:20] # Using smaller dataset for easier computation


In [ ]:
# English Sentence
print('English Sentence - ',english[1])
# Spanish Sentence
print('Spanish Sentence - ',spanish[1])

English Sentence -  Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries, where the survival rates for illnesses such as breast cancer can be half those of richer countries.
Spanish Sentence -  Los principales investigadores principales sostienen que esto puede permitir la detección precoz del cáncer, la tuberculosis, el VIH y la malaria en pacientes de países de bajos recursos, donde la tasa de supervivencia de enfermedades como el cáncer de mama puede ser la mitad de la de los países más avanzados.


## Data Statistics



In [ ]:

english_words = []
spanish_words = []
english_lengths = []
spanish_lengths = []
english_tokens = []
spanish_tokens = []
for i in range(len(english_100)):
  # Splitting the sentences into words
  words_en = english_100[i].split()
  words_sp = spanish_100[i].split()
  english_words.append(words_en)
  spanish_words.append(words_sp)

for i in range(len(english_words)):
  # Storing lenghts of all the sentences
  english_lengths.append(len(english_words[i]))
  spanish_lengths.append(len(spanish_words[i]))
  english_tokens.extend(english_words)
  spanish_tokens.extend(spanish_words)

# Storing all the unique words in the corpus to get word vocabulary
vocab_eng = []
vocab_spa = []
for i in range(len(english_tokens)):
  for token in english_tokens[i]:
    if token not in vocab_eng:
      vocab_eng.append(token)
  for token in spanish_tokens[i]:
    if token not in vocab_spa:
      vocab_spa.append(token)

# Printing data statistics
print('Minimum length English Se|ntence = ', np.min(english_lengths))
print('Maximum length English Sentence = ', np.max(english_lengths))
print('Average length English Sentence = ', np.mean(english_lengths))
print('English Vocabulary size = ',len(vocab_eng))
print('Minimum length Spanish Sentence = ', np.min(spanish_lengths))
print('Maximum length Spanish Sentence = ', np.max(spanish_lengths))
print('Average length Spanish Sentence = ', np.mean(spanish_lengths))
print('Spanish Vocabulary size = ',len(vocab_spa))


Minimum length English Se|ntence =  6
Maximum length English Sentence =  47
Average length English Sentence =  22.21
English Vocabulary size =  1237
Minimum length Spanish Sentence =  8
Maximum length Spanish Sentence =  60
Average length Spanish Sentence =  27.33
Spanish Vocabulary size =  1382


## Models
### OPUS-MT

In [ ]:
# Load the tokenizer and model from Hugging Face
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-es")

In [ ]:
# Tokenizing english sentences and padding them.
tokenized = tokenizer(english_20, return_tensors="pt", padding=True)
# Feeding the tokenized data into the model. Returns translated tokens
translated = model.generate(**tokenized)

In [ ]:
# Decoding the translated tokens using tokenizer class
output_opusmt = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

### M2M-100

In [ ]:
# Load the tokenizer and model from Hugging Face
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

In [ ]:
# Tokenizing english sentences and padding them.
tokenizer.src_lang = 'en'
tokenized = tokenizer(english_20, return_tensors="pt", padding=True)
# Feeding the tokenized data into the model. Returns translated tokens
translated = model.generate(**tokenized,forced_bos_token_id=tokenizer.get_lang_id('es'))

In [ ]:
# Decoding the translated tokens using tokenizer class
output_m2m100 = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

### MBART 50

In [ ]:
# Load the tokenizer and model from Hugging Face
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [ ]:
# Tokenizing english sentences and padding them.
tokenizer.src_lang = 'en_XX'
tokenized = tokenizer(english_20, return_tensors="pt", padding=True)
# Feeding the tokenized data into the model. Returns translated tokens
translated = model.generate(**tokenized, forced_bos_token_id=tokenizer.lang_code_to_id['es_XX'])


In [ ]:
# Decoding the translated tokens using tokenizer class
output_mbart50 = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

## Evaluation

In [58]:
# Functions for BLEU score.
def calculate_bleu(reference, hypothesis):
    bleu = sacrebleu.corpus_bleu(hypothesis, [reference])
    return bleu.score

In [59]:
# Passing the reference and translated sentence into the bleu function
opus_bleu = calculate_bleu(output_opusmt, spanish_20)
m2m_bleu = calculate_bleu(output_m2m100, spanish_20)
mbart_bleu = calculate_bleu(output_mbart50, spanish_20)

print('OPUS MT Bleu score = ', opus_bleu)
print('M2M 100 Bleu score = ', m2m_bleu)
print('MBART 50 Bleu score = ', mbart_bleu)


OPUS MT Bleu score =  34.33040263391583
M2M 100 Bleu score =  30.528845556510973
MBART 50 Bleu score =  27.503239179477948


## Comparison


In [ ]:
print('English Sentence : ',english_20[15])
print('Spanish Reference: ',spanish_20[15])
print('OPUSMT Translated: ',output_opusmt[15])
print('M2M 100 Translated:',output_m2m100[15])
print('MBART 50 Translated:',output_mbart50[15])



English Sentence :  "Panama Papers" is an umbrella term for roughly ten million documents from Panamanian law firm Mossack Fonseca, leaked to the press in spring 2016.
Spanish Reference:  La expresión genérica «Papeles de Panamá» se refiere a los casi diez millones de documentos del despacho de abogados panameño Mossack Fonseca que se filtraron en los medios en la primavera de 2016.
OPUSMT Translated:  "Panamá Papers" es un término general para aproximadamente diez millones de documentos del bufete de abogados panameño Mossack Fonseca, filtrados a la prensa en la primavera de 2016.
M2M 100 Translated: "Panama Papers" es un término de sombra para aproximadamente diez millones de documentos de la firma de ley de Panamá Mossack Fonseca, que se lanzó a la prensa en la primavera de 2016.
MBART 50 Translated: "Panama Papers" es un término de referencia para aproximadamente diez millones de documentos de la firma de abogados panameña Mossack Fonseca, que se difundieron a la prensa en primaver

## Conclusion
OPUS-MT : The bleu score while using OPUS MT was 34.28 making it the best model. The translations were well aligned with the reference data.
M2M-100 : The bleu score achieved was 30.50. Translations were not as good as OPUS MT. The execution was very slow and Colab runs out of memory.
MBART-50 : Worst performing model with bleu of 27.47.

Comparison between translations and reference:

OPUS MT translation has the same meaning but the structure of the sentence is different than the reference.

M2M 100 translation is also very close to OPUS MT translation with similar meaning with similar structure as OPUS MT. Although a few words were incorrect. For eg. OPUS MT - es un término general
M2M100 - es un término de sombra(sombra means shadow)

MBART 50 translation has more issues choosing the correct words compared to the other 2 models. The meaning of the sentence is not correct and the words like "de sombra" has been replaced with "de referencia" which has different meanings.


Due to out of memory issue, I had to reduce the size of data to just 20 sentences.